# KATO Prediction Quality Analysis

**Purpose**: Estimate text generation quality from training snapshot statistics WITHOUT running actual generation tests.

**This notebook analyzes:**
- ✅ **Zipf Quality**: Level-specific alpha targets (node0: 1.0-1.5, node1: 0.8-1.2, etc.)
- ✅ **Composition Quality**: Orphan rates and coverage metrics
- ✅ **Prediction Quality**: Fan-out and predictive_information scores
- ✅ **Hierarchical Consistency**: Frequency correlation validation
- ✅ **Overall HGR Score**: 0-100 Hierarchical Generation Readiness score

**Key Innovation**: Predict generation quality from pattern statistics alone, enabling:
- Configuration optimization before generation implementation
- Multi-run comparison and ranking
- Actionable recommendations for improvement

**Prerequisites**: 
- Completed training run with enhanced snapshot capture
- Snapshot file in `./snapshots/` directory

## 1. Setup and Imports

In [ ]:
from tools import (
    TrainingHistory,
    TrainingRunSnapshot,
)

from tools.prediction_quality_estimator import (
    compute_zipf_quality_score,
    compute_fanout_quality_score,
    compute_composition_quality_score,
    compute_hierarchical_consistency_score,
    compute_predictive_information_quality,
    compute_generation_readiness_score,
    compare_snapshots,
)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ All modules imported successfully")
print("✓ Ready for prediction quality analysis")

## 2. Load Training Snapshots

Load snapshots from previous training runs for analysis.

In [ ]:
# Initialize training history
history = TrainingHistory(db_path='./training_history.db', verbose=True)
history.print_summary()

# Get recent runs
recent_runs = history.get_recent_runs(n=10)

print(f"\n📋 Found {len(recent_runs)} recent training runs:")
for run in recent_runs:
    print(f"  - {run['run_id']}: {run['samples_processed']:,} samples, "
          f"chunk_size={run['config'].get('chunk_sizes', 'N/A')}")

In [ ]:
# Load snapshots for analysis
# Option 1: Load specific run by ID
# run_id = 'run_YYYYMMDD_HHMMSS'
# snapshot = TrainingRunSnapshot.load(f'./snapshots/{run_id}_snapshot.json')

# Option 2: Load most recent run
if recent_runs:
    latest_run = recent_runs[0]
    snapshot_path = f"./snapshots/{latest_run['run_id']}_snapshot.json"
    
    if Path(snapshot_path).exists():
        snapshot = TrainingRunSnapshot.load(snapshot_path)
        print(f"\n✓ Loaded snapshot: {snapshot.run_id}")
        print(f"  Total patterns: {snapshot.total_patterns:,}")
        print(f"  Node count: {len(snapshot.nodes)}")
    else:
        print(f"\n⚠️  Snapshot file not found: {snapshot_path}")
        print(f"  Please run training.ipynb Cell 24 to capture snapshot")
        snapshot = None
else:
    print("\n⚠️  No training runs found. Please run training.ipynb first.")
    snapshot = None

## 3. Pattern Coverage Analysis

Analyze how well patterns participate in the hierarchy:
- **Vocabulary richness**: unique_patterns / total_observations
- **Orphan rate**: % patterns with no parents
- **Coverage**: % patterns used by parent level

In [ ]:
if snapshot:
    print("\n" + "="*80)
    print("PATTERN COVERAGE ANALYSIS")
    print("="*80)
    
    node_names = sorted(snapshot.nodes.keys())
    
    # Collect coverage metrics
    coverage_data = []
    
    for node_name in node_names:
        ns = snapshot.nodes[node_name]
        comp_result = compute_composition_quality_score(snapshot, node_name)
        
        vocab_richness = ns.total_patterns / ns.total_observations if ns.total_observations > 0 else 0
        
        coverage_data.append({
            'node': node_name,
            'patterns': ns.total_patterns,
            'vocab_richness': vocab_richness,
            'orphan_rate': comp_result['orphan_rate'] or 0,
            'coverage': comp_result['coverage'] or 0,
            'status': comp_result['status']
        })
        
        print(f"\n{node_name}:")
        print(f"  Patterns: {ns.total_patterns:,}")
        print(f"  Vocab richness: {vocab_richness:.3f} (target: 0.3-0.7)")
        print(f"  Orphan rate: {comp_result['orphan_rate']:.1%} (target: <10%)")
        print(f"  Coverage: {comp_result['coverage']:.1%} (target: >70%)")
        print(f"  Quality: {comp_result['score']:.2f}/1.0 [{comp_result['status'].upper()}]")
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Vocabulary richness
    ax = axes[0]
    vocab_vals = [d['vocab_richness'] for d in coverage_data]
    ax.bar(range(len(node_names)), vocab_vals, color='skyblue', edgecolor='black')
    ax.axhline(0.3, color='green', linestyle='--', label='Target min', alpha=0.7)
    ax.axhline(0.7, color='green', linestyle='--', label='Target max', alpha=0.7)
    ax.set_xticks(range(len(node_names)))
    ax.set_xticklabels(node_names)
    ax.set_ylabel('Vocabulary Richness')
    ax.set_title('Vocabulary Richness by Level')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # Orphan rates
    ax = axes[1]
    orphan_vals = [d['orphan_rate'] * 100 for d in coverage_data]
    colors = ['red' if v > 30 else 'orange' if v > 20 else 'yellow' if v > 10 else 'green' 
              for v in orphan_vals]
    ax.bar(range(len(node_names)), orphan_vals, color=colors, edgecolor='black')
    ax.axhline(10, color='green', linestyle='--', label='Target (<10%)', alpha=0.7)
    ax.set_xticks(range(len(node_names)))
    ax.set_xticklabels(node_names)
    ax.set_ylabel('Orphan Rate (%)')
    ax.set_title('Orphan Rate by Level (Lower is Better)')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # Coverage
    ax = axes[2]
    coverage_vals = [d['coverage'] * 100 for d in coverage_data]
    colors = ['green' if v > 70 else 'yellow' if v > 50 else 'orange' if v > 30 else 'red'
              for v in coverage_vals]
    ax.bar(range(len(node_names)), coverage_vals, color=colors, edgecolor='black')
    ax.axhline(70, color='green', linestyle='--', label='Target (>70%)', alpha=0.7)
    ax.set_xticks(range(len(node_names)))
    ax.set_xticklabels(node_names)
    ax.set_ylabel('Coverage (%)')
    ax.set_title('Coverage by Level (Higher is Better)')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)

## 4. Frequency Distribution Quality

Analyze Zipfian power-law quality with **level-specific targets**:
- **node0**: α ≈ 1.0-1.5 (classic Zipfian)
- **node1**: α ≈ 0.8-1.2 (moderate power-law)
- **node2**: α ≈ 0.5-1.0 (weaker power-law)
- **node3**: α ≈ 0.2-0.5 (nearly uniform)

In [ ]:
if snapshot:
    print("\n" + "="*80)
    print("FREQUENCY DISTRIBUTION QUALITY")
    print("="*80)
    
    zipf_results = []
    
    for node_name in node_names:
        zipf_result = compute_zipf_quality_score(snapshot, node_name)
        zipf_results.append(zipf_result)
        
        if zipf_result['alpha'] is not None:
            print(f"\n{node_name}:")
            print(f"  Measured α: {zipf_result['alpha']:.3f}")
            print(f"  Target range: {zipf_result['target_range']}")
            print(f"  Ideal α: {zipf_result['ideal']:.2f}")
            print(f"  Deviation: {zipf_result['deviation']:.3f}")
            print(f"  Score: {zipf_result['score']:.2f}/1.0 [{zipf_result['status'].upper()}]")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Alpha values with target ranges
    ax = axes[0]
    alphas = [r['alpha'] for r in zipf_results if r['alpha'] is not None]
    ideals = [r['ideal'] for r in zipf_results if r['alpha'] is not None]
    mins = [r['target_range'][0] for r in zipf_results if r['alpha'] is not None]
    maxs = [r['target_range'][1] for r in zipf_results if r['alpha'] is not None]
    valid_nodes = [node_names[i] for i, r in enumerate(zipf_results) if r['alpha'] is not None]
    
    x_pos = range(len(valid_nodes))
    ax.plot(x_pos, alphas, 'o-', color='blue', markersize=10, linewidth=2, label='Measured α')
    ax.plot(x_pos, ideals, 's--', color='green', markersize=8, linewidth=1.5, label='Ideal α')
    ax.fill_between(x_pos, mins, maxs, alpha=0.2, color='green', label='Target range')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(valid_nodes)
    ax.set_ylabel('Zipfian Alpha (α)')
    ax.set_title('Zipfian Alpha by Level')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Quality scores
    ax = axes[1]
    scores = [r['score'] * 100 for r in zipf_results if r['alpha'] is not None]
    colors = ['green' if s > 80 else 'yellow' if s > 60 else 'orange' if s > 40 else 'red'
              for s in scores]
    ax.bar(x_pos, scores, color=colors, edgecolor='black')
    ax.axhline(80, color='green', linestyle='--', alpha=0.7, label='Excellent (>80)')
    ax.axhline(60, color='yellow', linestyle='--', alpha=0.7, label='Good (>60)')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(valid_nodes)
    ax.set_ylabel('Quality Score')
    ax.set_title('Zipf Quality Score by Level')
    ax.set_ylim(0, 100)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)

## 5. Hierarchical Consistency

Validate that parent frequencies correlate with sum of child frequencies.

**Expected**: Strong positive correlation (>0.7)
- If parent appears 10 times, its children should sum to ~10 occurrences
- Low correlation indicates structural issues

In [ ]:
if snapshot:
    print("\n" + "="*80)
    print("HIERARCHICAL CONSISTENCY")
    print("="*80)
    
    consistency_results = []
    
    # Skip node0 (no children)
    for node_name in node_names[1:]:
        consistency_result = compute_hierarchical_consistency_score(snapshot, node_name)
        consistency_results.append((node_name, consistency_result))
        
        print(f"\n{node_name} vs {node_names[node_names.index(node_name) - 1]}:")
        if consistency_result['freq_correlation'] is not None:
            print(f"  Frequency correlation: {consistency_result['freq_correlation']:.3f} (target: >0.7)")
            print(f"  Score: {consistency_result['score']:.2f}/1.0 [{consistency_result['status'].upper()}]")
        else:
            print(f"  Frequency correlation: N/A (insufficient data)")
        
        if consistency_result['freq_compression_ratio'] is not None:
            print(f"  Compression ratio: {consistency_result['freq_compression_ratio']:.2f}x")
    
    # Visualize
    valid_results = [(n, r) for n, r in consistency_results if r['freq_correlation'] is not None]
    
    if valid_results:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Correlation values
        ax = axes[0]
        nodes_list = [n for n, r in valid_results]
        corrs = [r['freq_correlation'] for n, r in valid_results]
        colors = ['green' if c >= 0.7 else 'yellow' if c >= 0.5 else 'orange' if c >= 0.3 else 'red'
                  for c in corrs]
        ax.bar(range(len(nodes_list)), corrs, color=colors, edgecolor='black')
        ax.axhline(0.7, color='green', linestyle='--', alpha=0.7, label='Target (>0.7)')
        ax.set_xticks(range(len(nodes_list)))
        ax.set_xticklabels(nodes_list)
        ax.set_ylabel('Spearman Correlation')
        ax.set_title('Parent-Child Frequency Correlation')
        ax.set_ylim(-0.1, 1.0)
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        
        # Quality scores
        ax = axes[1]
        scores = [r['score'] * 100 for n, r in valid_results]
        colors = ['green' if s > 80 else 'yellow' if s > 60 else 'orange' if s > 40 else 'red'
                  for s in scores]
        ax.bar(range(len(nodes_list)), scores, color=colors, edgecolor='black')
        ax.axhline(80, color='green', linestyle='--', alpha=0.7, label='Excellent (>80)')
        ax.set_xticks(range(len(nodes_list)))
        ax.set_xticklabels(nodes_list)
        ax.set_ylabel('Quality Score')
        ax.set_title('Consistency Quality Score')
        ax.set_ylim(0, 100)
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\n" + "="*80)

## 6. Prediction Quality Estimation

Analyze sampled predictions to estimate generation quality:
- **Predictive information**: How reliably patterns predict their futures
- **Potential scores**: similarity × predictive_information
- **Fan-out**: Number of predictions per query (level-specific targets)

In [ ]:
if snapshot:
    print("\n" + "="*80)
    print("PREDICTION QUALITY ESTIMATION")
    print("="*80)
    
    # Check if prediction samples were captured
    has_predictions = any(
        snapshot.nodes[n].prediction_samples is not None 
        for n in node_names
    )
    
    if has_predictions:
        for node_name in node_names:
            ns = snapshot.nodes[node_name]
            
            if ns.prediction_samples:
                print(f"\n{node_name}:")
                
                # Predictive information
                pi_result = compute_predictive_information_quality(snapshot, node_name)
                if pi_result['mean_pi'] is not None:
                    print(f"  Predictive information: {pi_result['mean_pi']:.3f} (target: >0.5)")
                    print(f"  Mean potential: {pi_result['mean_potential']:.3f}")
                    print(f"  PI Quality: [{pi_result['status'].upper()}]")
                
                # Fan-out
                fanout_result = compute_fanout_quality_score(snapshot, node_name)
                if fanout_result['mean_fanout'] is not None:
                    print(f"  Mean fan-out: {fanout_result['mean_fanout']:.1f}")
                    print(f"  Target range: {fanout_result['target_range']}")
                    print(f"  Fan-out Quality: [{fanout_result['status'].upper()}]")
        
        # Visualize
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Predictive information
        ax = axes[0]
        pi_vals = []
        valid_nodes_pi = []
        for node_name in node_names:
            pi_result = compute_predictive_information_quality(snapshot, node_name)
            if pi_result['mean_pi'] is not None:
                pi_vals.append(pi_result['mean_pi'])
                valid_nodes_pi.append(node_name)
        
        if pi_vals:
            ax.bar(range(len(valid_nodes_pi)), pi_vals, color='skyblue', edgecolor='black')
            ax.axhline(0.5, color='green', linestyle='--', alpha=0.7, label='Target (>0.5)')
            ax.set_xticks(range(len(valid_nodes_pi)))
            ax.set_xticklabels(valid_nodes_pi)
            ax.set_ylabel('Mean Predictive Information')
            ax.set_title('Predictive Information by Level')
            ax.set_ylim(0, 1.0)
            ax.legend()
            ax.grid(axis='y', alpha=0.3)
        
        # Fan-out
        ax = axes[1]
        fanout_vals = []
        valid_nodes_fo = []
        for node_name in node_names:
            fo_result = compute_fanout_quality_score(snapshot, node_name)
            if fo_result['mean_fanout'] is not None:
                fanout_vals.append(fo_result['mean_fanout'])
                valid_nodes_fo.append(node_name)
        
        if fanout_vals:
            colors = ['green' if 50 < v < 150 else 'yellow' for v in fanout_vals]
            ax.bar(range(len(valid_nodes_fo)), fanout_vals, color=colors, edgecolor='black')
            ax.set_xticks(range(len(valid_nodes_fo)))
            ax.set_xticklabels(valid_nodes_fo)
            ax.set_ylabel('Mean Fan-out')
            ax.set_title('Prediction Fan-out by Level')
            ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("\n⚠️  No prediction samples found in snapshot.")
        print("   Snapshot was captured without prediction sampling.")
        print("   Re-run training.ipynb Cell 24 with capture_prediction_samples=True")
    
    print("\n" + "="*80)

## 7. Hierarchical Generation Readiness (HGR) Score

**Overall quality score (0-100)** that predicts text generation quality.

**Categories**:
1. Zipf Quality (25%)
2. Composition Quality (25%)
3. Prediction Quality (25%)
4. Hierarchical Consistency (20%)
5. Diversity (5%)

**Interpretation**:
- **90-100**: Excellent - ready for generation
- **70-89**: Good - minor improvements possible
- **50-69**: Warning - review specific categories
- **30-49**: Poor - significant issues
- **0-29**: Critical - not ready for generation

In [ ]:
if snapshot:
    print("\n" + "="*80)
    print("HIERARCHICAL GENERATION READINESS (HGR) SCORE")
    print("="*80 + "\n")
    
    hgr_result = compute_generation_readiness_score(snapshot)
    
    # Overall score
    print(f"🎯 OVERALL HGR SCORE: {hgr_result['overall_score']:.1f}/100")
    print(f"   Health Status: {hgr_result['health_status']}\n")
    
    # Category breakdown
    print("📊 CATEGORY SCORES:")
    for category, score in hgr_result['category_scores'].items():
        score_pct = score * 100
        status_icon = '🟢' if score > 0.8 else '🟡' if score > 0.6 else '🟠' if score > 0.4 else '🔴'
        print(f"  {status_icon} {category.replace('_', ' ').title()}: {score_pct:.1f}/100")
    
    # Recommendations
    print("\n💡 RECOMMENDATIONS:")
    for i, rec in enumerate(hgr_result['recommendations'], 1):
        print(f"  {i}. {rec}")
    
    # Visualize category scores
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Radar chart
    ax = axes[0]
    categories = list(hgr_result['category_scores'].keys())
    values = [hgr_result['category_scores'][c] * 100 for c in categories]
    
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    values += values[:1]  # Complete the circle
    angles += angles[:1]
    
    ax = plt.subplot(121, projection='polar')
    ax.plot(angles, values, 'o-', linewidth=2, color='blue')
    ax.fill(angles, values, alpha=0.25, color='blue')
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([c.replace('_', '\n').title() for c in categories], size=8)
    ax.set_ylim(0, 100)
    ax.set_title('HGR Category Breakdown', pad=20)
    ax.grid(True)
    
    # Bar chart
    ax = axes[1]
    colors = ['green' if v > 80 else 'yellow' if v > 60 else 'orange' if v > 40 else 'red'
              for v in values[:-1]]
    ax.barh(range(len(categories)), values[:-1], color=colors, edgecolor='black')
    ax.set_yticks(range(len(categories)))
    ax.set_yticklabels([c.replace('_', ' ').title() for c in categories])
    ax.set_xlabel('Score')
    ax.set_xlim(0, 100)
    ax.axvline(80, color='green', linestyle='--', alpha=0.5, label='Excellent')
    ax.axvline(60, color='yellow', linestyle='--', alpha=0.5, label='Good')
    ax.set_title('Category Scores')
    ax.legend()
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*80)

## 8. Multi-Run Comparison

Compare multiple training runs to identify optimal configurations.

In [ ]:
# Load all available snapshots
snapshot_dir = Path('./snapshots')
snapshot_files = list(snapshot_dir.glob('*_snapshot.json'))

print(f"Found {len(snapshot_files)} snapshot files\n")

if len(snapshot_files) > 1:
    # Load all snapshots
    snapshots = []
    for snapshot_file in snapshot_files:
        try:
            s = TrainingRunSnapshot.load(str(snapshot_file))
            snapshots.append(s)
        except Exception as e:
            print(f"⚠️  Failed to load {snapshot_file.name}: {e}")
    
    if len(snapshots) > 1:
        # Compare snapshots
        comparison = compare_snapshots(snapshots, metric='overall_score')
        
        print("="*80)
        print("MULTI-RUN COMPARISON")
        print("="*80 + "\n")
        
        print("📊 RANKINGS BY HGR SCORE:\n")
        for i, (run_id, score) in enumerate(comparison['rankings'], 1):
            medal = '🥇' if i == 1 else '🥈' if i == 2 else '🥉' if i == 3 else '  '
            print(f"{medal} {i}. {run_id}: {score:.1f}/100")
        
        print(f"\n🏆 Best configuration: {comparison['best_run']}")
        print(f"   Score: {comparison['rankings'][0][1]:.1f}/100")
        
        # Visualize comparison
        fig, ax = plt.subplots(figsize=(12, 6))
        
        run_ids = [rid for rid, _ in comparison['rankings']]
        scores = [score for _, score in comparison['rankings']]
        colors = ['green' if s >= 80 else 'yellow' if s >= 60 else 'orange' if s >= 40 else 'red'
                  for s in scores]
        
        ax.barh(range(len(run_ids)), scores, color=colors, edgecolor='black')
        ax.set_yticks(range(len(run_ids)))
        ax.set_yticklabels([rid[-15:] for rid in run_ids])  # Show last 15 chars
        ax.set_xlabel('HGR Score')
        ax.set_title('Training Run Comparison (Ranked by HGR Score)')
        ax.set_xlim(0, 100)
        ax.axvline(80, color='green', linestyle='--', alpha=0.5, label='Excellent')
        ax.axvline(60, color='yellow', linestyle='--', alpha=0.5, label='Good')
        ax.legend()
        ax.grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("\n" + "="*80)
    else:
        print("Only one valid snapshot loaded. Need at least 2 for comparison.")
else:
    print("Only one snapshot found. Run more training sessions for comparison.")

## 9. Detailed Metrics Export

Export comprehensive metrics table for external analysis.

In [ ]:
if snapshot:
    # Build comprehensive metrics table
    metrics_table = []
    
    for node_name in node_names:
        ns = snapshot.nodes[node_name]
        
        zipf_result = compute_zipf_quality_score(snapshot, node_name)
        comp_result = compute_composition_quality_score(snapshot, node_name)
        fanout_result = compute_fanout_quality_score(snapshot, node_name)
        pi_result = compute_predictive_information_quality(snapshot, node_name)
        
        metrics_table.append({
            'Node': node_name,
            'Patterns': ns.total_patterns,
            'Zipf_Alpha': zipf_result['alpha'],
            'Zipf_Score': zipf_result['score'],
            'Orphan_Rate': comp_result['orphan_rate'],
            'Coverage': comp_result['coverage'],
            'Comp_Score': comp_result['score'],
            'Mean_Fanout': fanout_result.get('mean_fanout'),
            'Fanout_Score': fanout_result['score'],
            'Mean_PI': pi_result.get('mean_pi'),
            'PI_Score': pi_result['score']
        })
    
    df = pd.DataFrame(metrics_table)
    
    print("\n" + "="*80)
    print("COMPREHENSIVE METRICS TABLE")
    print("="*80 + "\n")
    print(df.to_string(index=False))
    
    # Export to CSV
    csv_path = f'./metrics/{snapshot.run_id}_metrics.csv'
    Path('./metrics').mkdir(exist_ok=True)
    df.to_csv(csv_path, index=False)
    print(f"\n✓ Exported to {csv_path}")
    
    print("\n" + "="*80)

## 10. Interpretation Guide

### Understanding HGR Scores

**Overall HGR Score Ranges:**
- **90-100 (EXCELLENT)**: System is ready for text generation. All metrics in healthy ranges.
- **70-89 (GOOD)**: Minor improvements possible, but generation should work well.
- **50-69 (WARNING)**: Review weak categories. Generation may have quality issues.
- **30-49 (POOR)**: Significant problems detected. Address before generation.
- **0-29 (CRITICAL)**: Not ready for generation. Major configuration issues.

### Category Interpretations

**1. Zipf Quality (25%):**
- Measures if frequency distributions follow expected power-laws
- Different targets per level (node0: 1.0-1.5, node1: 0.8-1.2, etc.)
- Low score → Adjust chunk_size or increase training data

**2. Composition Quality (25%):**
- Orphan rate: % patterns not used by higher levels
- Coverage: % patterns participating in hierarchy
- Low score → Increase training samples or review segmentation

**3. Prediction Quality (25%):**
- Predictive information: Pattern reliability
- Fan-out: Number of predictions (level-specific targets)
- Low score → Adjust recall_threshold or improve pattern diversity

**4. Hierarchical Consistency (20%):**
- Frequency correlation: Parent freq vs sum(child freqs)
- Target: >0.7 correlation
- Low score → Review hierarchy architecture

**5. Diversity (5%):**
- Vocabulary richness
- Target: 0.3-0.7 (Goldilocks zone)

### Common Issues & Solutions

**Issue**: Low Zipf quality at node0
- **Solution**: Train on more samples (10K → 100K) or adjust chunk_size

**Issue**: High orphan rate (>30%)
- **Solution**: Increase training data or review document boundaries

**Issue**: Low coverage (<50%)
- **Solution**: Patterns not being composed effectively; increase chunk_size

**Issue**: Poor frequency correlation (<0.5)
- **Solution**: Structural issues; review hierarchy design

**Issue**: Fan-out too low (<5) or too high (>100)
- **Solution**: Adjust recall_threshold in KATO configuration

### Next Steps Based on HGR Score

**If HGR > 70:**
1. System is ready for generation experiments
2. Consider implementing text generation module
3. Validate HGR predictions with actual generation tests

**If 50 < HGR < 70:**
1. Review category breakdowns
2. Address weak categories with specific solutions
3. Re-train and re-analyze
4. Target HGR > 70 before generation

**If HGR < 50:**
1. Fundamental issues detected
2. Review all recommendations carefully
3. Consider different configuration:
   - Try different chunk_sizes
   - Increase training data significantly
   - Review data quality and segmentation
4. Re-train from scratch with new configuration

### Validation (Future Work)

To validate HGR predictions:
1. Implement text generation module
2. Generate test outputs from multiple configurations
3. Measure actual generation quality (coherence, diversity, faithfulness)
4. Correlate HGR scores with actual quality
5. Refine metric weights if needed

**Success criterion**: Pearson correlation > 0.7 between HGR and actual quality